In [1]:
import mysql.connector
import pandas as pd
import numpy as np

In [8]:
engine = mysql.connector.connect(
    host="music.c9falgyyzaqw.ap-south-1.rds.amazonaws.com",
    username = "hari31416",
    password = "HariSQL123",
    database = "music"
)

In [9]:
cursor = engine.cursor()

In [10]:
cursor.execute("SELECT * FROM songs LIMIT 10")

In [11]:
engine.disconnect()